In [1]:
class entity:
    def __init__(self, recipe):
        self.recipe = recipe
        self.ingredient = [x for x in recipe["food_ingredients"]]
        self.number = [0.0 for x in recipe["food_ingredients"]]
        self.unit = ["TBD" for x in recipe["food_ingredients"]]
        self.entities = ["TBD" for x in recipe["food_ingredients"]]
#         self.failure
#         self.recipe_triple = []
    def get_nlpres(self, nlp_output, index):
        if not nlp_output:
            self.number[index]="TBD"
            self.unit[index] = "TBD"
            self.entities[index] = "TBD"
#             print("")
            return
        res = 0.0
        for v in nlp_output:
            if v[1] != 'CARDINAL' and v[1] != "QUANTITY":
#                 print(v)
                pass
            elif v[1] == "CARDINAL":
                vals = v[0].split()
#                 res = 0.0
                for _ in vals:
                    try:
                        res+=eval(_)
                    except:
                        break
                self.number[index]=res
        
#                 self.unit.append("TBD")
#                 self.entities.append("TBD")
            else:
                vals = v[0].split()
#                 res = 0.0
                for _ in vals:
                    try:
                        res += eval(_)
                    except:
                        self.unit[index] = _
                        break
                self.number[index] = res
#                 self.entities.append("TBD")
            #TBD as default since cannot find

In [9]:
class food_entity:
    def __init__(self, topclass, subclass, name, price, unit_price):
        self.name = name
        self.topclass = topclass
        self.subclass = subclass
        self.price = price
        self.unit_price = unit_price
        self.unit = unit_price.split("/")[-1]
    
    def __str__(self):
        return str((self.topclass, self.name, self.subclass, self.price, self.unit_price))

In [36]:
import pickle

In [37]:
with open("./food.pkl", "rb") as f:
    all_food = pickle.load(f)
with open("./final_round_extraction.pkl", "rb") as f:
    all_recipes = pickle.load(f)

In [38]:
len(all_food)

17263

In [39]:
len(all_recipes)

3828

### solving units in all_recipes

In [40]:
recipe_units = {'DEFAULT',
 'cube',
 'cup',
 'gallon',
 'gram',
 'inch',
 'liter',
 'milliliter',
 'not valid',
 'ounce',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}

In [55]:
food_unit = set()
for v in all_food:
    food_unit.add(v.unit)

In [56]:
recipe_unit = set()
for r in all_recipes:
    for index in range(len(r.unit)):
        recipe_unit.add(r.unit[index])

In [57]:
recipe_unit

{'DEFAULT',
 'gram',
 'liter',
 'not valid',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}

In [58]:
food_unit

{'DEFAULT',
 'gram',
 'kg',
 'liter',
 'ounce',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}

In [47]:
mapper = {"cube": ("DEFAULT", 1),
          "DEFAULT":("DEFAULT", 1),
          "cup":("liter", 0.5),
          "gallon": ("liter", 3.78541),
          "gram":("gram", 1),
          "inch":("DEFAULT", 1),
          "liter":("liter", 1),
          "milliliter":("liter", 0.001),
          "ounce":("ounce", 1),
          "oz":("oz", 1),
          "pound":("pound", 1),
          "quart":("quart", 1), 
          "tablespoon":("tablespoon", 1), 
          "teaspoon":("teaspoon", 1)}


In [52]:
for r in all_recipes:
    for index in range(len(r.unit)):
        if r.unit[index]!= "not valid":
            r.number[index] *= mapper[r.unit[index]][1]
            r.unit[index] = mapper[r.unit[index]][0]

In [53]:
all_recipes[0].number

[0.625, 0.3333333333333333, 1.0, 1.0, 0.125, 0.25, 1.0, 0.5]

In [54]:
all_recipes[0].unit

['liter',
 'liter',
 'teaspoon',
 'liter',
 'teaspoon',
 'liter',
 'liter',
 'DEFAULT']

In [51]:
all_recipes[0].entities

['butter',
 'white sugar',
 'vanilla extract',
 'purpose flour',
 'salt',
 'unsweetened cocoa powder',
 'chop pecan',
 'confectioner sugar decoration']

In [59]:
print(all_food[0])

('nectarine', 'Red Seedless Grapes, 2 lb', 'nectarine', '3.51', '0.11')


In [62]:
all_recipes[0].recipe

{'food_url': 'https://www.allrecipes.com/recipe/15253/chocolate-snowballs/',
 'type_url': 'https://www.allrecipes.com/recipes/841/holidays-and-events/christmas/desserts/christmas-cookies/',
 'type_name': 'Christmas Cookies',
 'food_name': 'Chocolate Snowballs',
 'food_ingredients': ['1 1/4 cups butter',
  '2/3 cup white sugar',
  '1 teaspoon vanilla extract',
  '2 cups all-purpose flour',
  '1/8 teaspoon salt',
  '1/2 cup unsweetened cocoa powder',
  '2 cups chopped pecans',
  "1/2 cup confectioners' sugar for decoration"]}

### begin entity linking
1. define ontology
2. by calculating distance on different items to choose the best similarity 

## Ontology
_ a schema:recipe;

    rdfs:value recipe_name
    
    schema:url url
    
    schema:foodtype recipe_type
    
    schema:ingredient ingredient
    
_ a schema:food;

    rdfs:value food_name
    
    schema:price price
    
    schema:unit unit
    
    schema:unit_price unit_price
    
_ a schema:usedby;
    
    rdfs:domain food
    
    rdfs:range recipe
    
    schema:amount number
    
    schema:unit unit
    
    schema:unit_price unit price

In [75]:
from collections import defaultdict

In [105]:
grouped_food  = defaultdict(list)
cnt = 0
for v in all_food:
    grouped_food[v.topclass].append((v, cnt))
    cnt += 1

In [69]:
from rltk.similarity.jaro import jaro_winkler_distance

In [97]:
not_joined = set() # set to NULL for this class when adding entity
all_set = set()
for r in all_recipes:
    for index in range(len(r.unit)):
        all_set.add(r.entities[index])
        if r.entities[index] not in grouped_food:
            not_joined.add(r.entities[index])

In [98]:
len(all_set)

3964

In [102]:
len(not_joined)

134

In [99]:
lenset = set()
checkset = defaultdict(set)
for r in all_recipes:
    for index in range(len(r.unit)):
        lenset.add(len(r.entities[index].split()))
        checkset[len(r.entities[index].split())].add(r.entities[index])

In [100]:
print(len(checkset[1])+len(checkset[2])+len(checkset[3])+len(checkset[4])+len(checkset[5])+len(checkset[6]))

3964


In [101]:
print(len(grouped_food))

3838


In [96]:
print(len(one)+len(two)+len(three)+len(four)+len(five)+len(six))

3973


In [77]:
one  = set()
with open("./1.txt", "r") as f:
    for line in f:
        one.add(line.strip())
two = set()
with open("./2.txt", "r") as f:
    for line in f:
        two.add(line.strip())
three = set()
with open("./3.txt", "r") as f:
    for line in f:
        three.add(line.strip())
four = set()
with open("./4.txt", "r") as f:
    for line in f:
        four.add(line.strip())
five = set()
with open("./5.txt", "r") as f:
    for line in f:
        five.add(line.strip())
six = set()
with open("./6.txt", "r") as f:
    for line in f:
        six.add(line.strip())

In [87]:
checkset[6].difference(six)

set()

In [113]:
import random

1.866714223681405

In [162]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS, XSD, NamespaceManager
SCHEMA = Namespace("http://schema.org/")
namespace_manager = NamespaceManager(Graph())
namespace_manager.bind("schema", SCHEMA, override=False)

ontologies = Graph()
ontologies.namespace_manager = namespace_manager

In [163]:
def resolving_url(url):
    return url.replace(".", "-").replace(":", "-").replace("/", "-")

In [164]:
for r in all_recipes:
    tmp = BNode(resolving_url(r.recipe["food_url"]))
    ontologies.add((tmp, RDF.type, SCHEMA.recipe))
    ontologies.set((tmp, RDF.value, Literal(str(r.recipe["food_name"]), datatype=XSD.string)))
    ontologies.set((tmp, SCHEMA.recipe_url, Literal(r.recipe["food_url"])))
    ontologies.set((tmp, SCHEMA.type_url, Literal(r.recipe["type_url"])))
    ontologies.set((tmp, SCHEMA.ingredient, Literal(str(r.recipe["food_ingredients"]), datatype=XSD.string)))
#     for index, value in enumerate(r.entities): # defined food used by
#         if value not in not_joined:
#             possible_solution = 
            
#         else:
#         self.name = name
#         self.topclass = topclass
#         self.subclass = subclass
#         self.price = price
#         self.unit_price = unit_price
#         self.unit = unit_price.split("/")[-1]
for key, value in grouped_food.items():
    for v in value:
        tmp = BNode(v[1])
        ontologies.add((tmp, RDF.type, SCHEMA.food))
        ontologies.set((tmp, RDF.value, Literal(str(v[0].name), datatype=XSD.string)))
        ontologies.set((tmp, SCHEMA.topclass, Literal(v[0].topclass)))
        ontologies.set((tmp, SCHEMA.subclass, Literal(v[0].subclass)))
        try:
            ontologies.set((tmp, SCHEMA.price, Literal(float(v[0].price), datatype=XSD.double)))
        except:
            print("random price")
            ontologies.set((tmp, SCHEMA.price, Literal(random.uniform(2.0, 10.0), datatype=XSD.double)))
        try:
            ontologies.set((tmp, SCHEMA.unit_price, Literal(float(v[0].unit_price), datatype=XSD.double)))
        except:
            print("random unit_price")
            ontologies.set((tmp, SCHEMA.unit_price, Literal(random.uniform(1.0, 2.0), datatype=XSD.double)))
        ontologies.set((tmp, SCHEMA.unit, Literal(v[0].unit)))

random price
random unit_price
random price
random unit_price
random price
random unit_price
random price
random unit_price
random price
random unit_price
random price
random unit_price
random price
random unit_price
random price
random unit_price


In [165]:
from uuid import uuid4

In [166]:
# usedby
for r in all_recipes:
    for index, value in enumerate(r.entities):
        if value != "not valid":
            food_candidates = grouped_food[value]
            if food_candidates:
                food_candidates.sort(key=lambda x: jaro_winkler_distance(str(x[0].name), value), reverse=True)
    #             print(food_candidates[0][0].name, value)
                using_food = food_candidates[0]
                recipe = BNode(resolving_url(r.recipe["food_url"]))
                food = BNode(using_food[1])
                relation = BNode(uuid4())
                ontologies.add((relation, RDF.type, SCHEMA.usedby))
                ontologies.set((relation, SCHEMA.inrecipe, recipe))
                ontologies.set((relation, SCHEMA.forfood, food))
                ontologies.set((relation, SCHEMA.amount, Literal(float(r.number[index]), datatype=XSD.double)))
                ontologies.set((relation, SCHEMA.unit, Literal(str(r.unit[index]))))
                try:
                    ontologies.set((relation, SCHEMA.unit_price, Literal(float(v[0].unit_price), datatype=XSD.double)))
                except:
                    print("random unit_price")
                    ontologies.set((relation, SCHEMA.unit_price, Literal(random.uniform(2.0, 10.0), datatype=XSD.double)))



In [167]:
ontologies.serialize(destination="recipe_food.ttl" ,format='turtle')

In [138]:
for g in grouped_food:
    if not grouped_food[g]:
        print(g)

multicolored candy sprinkle optional


3838